In [1]:
#Creating Meta data 
import sys, os, datetime
import getpass
import pkg_resources
import platform
import socket  

In [2]:
# Importing libraries
import sys
import matplotlib as mpl 
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd
import pathlib
import warnings
warnings.filterwarnings("ignore")

# Deffining directories
current_dir = pathlib.Path().absolute()
fa_dir = current_dir.parents[0].joinpath('src')

# Importing fish_analyses module
sys.path.append(str(fa_dir))
import fish_analyses as fa


data_dir = current_dir.parents[0].joinpath('dataBases').joinpath('GAPDH_Exon_Cy5')     # path to a folder with images.
channels_with_cytosol = [1,2]            # list or int indicating the channels where the cytosol is detectable
channels_with_nucleus = 0                # list or int indicating the channels where the nucleus is detectable
channels_with_FISH = [1,2]               # list or int with the channels with FISH spots that are used for the quantification
diamter_nucleus = 200                    # approximate nucleus size in pixels
diameter_cytosol = 250 #250              # approximate cytosol size in pixels
# Parameters for FISH detection
voxel_size_z = 500                       # Microscope conversion px to nanometers in the z axis.
voxel_size_yx = 103                      # Microscope conversion px to nanometers in the xy axis.
psf_z = 600      #350                    # Theoretical size of the PSF emitted by a [rna] spot in the z plan, in nanometers.
psf_yx = 150     #150                    # Theoretical size of the PSF emitted by a [rna] spot in the yx plan, in nanometers.
# Cluster Detection
minimum_spots_cluster = 7                # The number of spots in a neighborhood for a point to be considered as a core point (from which a cluster is expanded). This includes the point itself.
show_plots=True                          # Flag to display plots

2021-11-08 21:41:03,235 [INFO] WRITING LOG OUTPUT TO /Users/luisaguilera/.cellpose/run.log


In [ ]:
# dataframe_FISH = fa.PipelineFISH(data_dir, channels_with_cytosol, channels_with_nucleus, channels_with_FISH,diamter_nucleus, diameter_cytosol, voxel_size_z, voxel_size_yx, psf_z, psf_yx, minimum_spots_cluster,show_plot=show_plots).run()

In [ ]:
class Metadata():
    '''
    This class is intended to generate a metadata file with information about used dependencies, user information, and parameters used.
    
    Parameters
    --  --  --  --  -- 
    parameter: bool, optional
        parameter description. The default is True. 
    '''
    def __init__(self,data_dir, channels_with_cytosol, channels_with_nucleus, channels_with_FISH,diamter_nucleus, diameter_cytosol, voxel_size_z, voxel_size_yx, psf_z, psf_yx, minimum_spots_cluster):
        self.list_images, self.path_files, self.list_files_names, self.number_images = ReadImages(data_dir).read()
        self.channels_with_cytosol = channels_with_cytosol
        self.channels_with_nucleus = channels_with_nucleus
        self.channels_with_FISH = channels_with_FISH
        self.diamter_nucleus = diamter_nucleus
        self.diameter_cytosol = diameter_cytosol
        #self.microscope_px_to_nm = microscope_px_to_nm
        #self.microscope_psf = microscope_psf
        self.voxel_size_z = voxel_size_z
        self.voxel_size_yx = voxel_size_yx 
        self.psf_z = psf_z
        self.psf_yx = psf_yx
        self.minimum_spots_cluster = minimum_spots_cluster
        self.CLUSTER_RADIUS = 500
        
        self.filename = './metadata_'+ str(data_dir.name) +'.txt'
        
    def write_metadata(self):
      installed_modules = [str(module).replace(" ","==") for module in pkg_resources.working_set]
      important_modules = [ 'tqdm', 'torch','tifffile', 'setuptools', 'scipy', 'scikit-learn', 'scikit-image', 'PyYAML', 'pysmb', 'pyfiglet', 'pip', 'Pillow', 'pandas', 'opencv-python-headless', 'numpy', 'numba', 'natsort', 'mrc', 'matplotlib', 'llvmlite', 'jupyter-core', 'jupyter-client', 'joblib', 'ipython', 'ipython-genutils', 'ipykernel', 'cellpose', 'big-fish']
      
      def create_data_file(filename):
        if sys.platform == 'linux' or sys.platform == 'darwin':
          os.system('touch' + filename)
        elif sys.platform == 'win32':
          os.system('echo , > ' + filename)
          
      number_spaces_pound_sign = 65
      def write_data_in_file(filename):
          with open(filename, 'w') as fd:
              fd.write('#' * (number_spaces_pound_sign)) 
              fd.write('\n#       AUTHOR INFORMATION  ')
              fd.write('\n Author: ' + getpass.getuser())
              fd.write('\n Created at: ' + datetime.datetime.today().strftime('%d %b %Y'))
              fd.write('\n Time: ' + str(datetime.datetime.now().hour) + ':' + str(datetime.datetime.now().minute) )
              fd.write('\n Operative System: ' + sys.platform )
              fd.write('\n hostname: ' + socket.gethostname() + '\n')
              
              fd.write('#' * (number_spaces_pound_sign) ) 
              fd.write('\n#       PARAMETERS USED  ')
              
              fd.write('\n    channels_with_cytosol: ' + str(self.channels_with_cytosol) )
              fd.write('\n    channels_with_nucleus: ' + str(self.channels_with_nucleus) )
              fd.write('\n    channels_with_FISH: ' + str(self.channels_with_FISH) )
              fd.write('\n    diamter_nucleus: ' + str(self.diamter_nucleus) )
              fd.write('\n    diameter_cytosol: ' + str(self.diameter_cytosol) )
              fd.write('\n    voxel_size_z: ' + str(self.voxel_size_z) )
              fd.write('\n    voxel_size_yx: ' + str(self.voxel_size_yx) )
              fd.write('\n    psf_z: ' + str(self.psf_z) )
              fd.write('\n    psf_yx: ' + str(self.psf_yx) )
              fd.write('\n    minimum_spots_cluster: ' + str(self.minimum_spots_cluster) )
              fd.write('\n') 
              fd.write('#' * (number_spaces_pound_sign) ) 
              fd.write('\n#       FILES AND DIRECTORIES USED ')
              fd.write('\n Directory path: ' + str(self.data_dir) )
              fd.write('\n Folder name: ' + str(self.data_dir.name)  )
              # for loop for all the images.
              fd.write('\n Images in path :'  )
              
              for img_name in self.list_files_names:
                fd.write('\n    '+ img_name)
              fd.write('\n')  
              
              fd.write('#' * (number_spaces_pound_sign)) 
              fd.write('\n       REPRODUCIBILITY ')
              fd.write('\n Platform: \n')
              fd.write('    Python: ' + str(platform.python_version()) )
              fd.write('\n Dependancies: ')

              # iterating for all modules
              for module_name in installed_modules:
                if any(module_name[0:4] in s for s in important_modules):
                  fd.write('\n    '+ module_name)
              fd.write('\n') 
              fd.write('#' * (number_spaces_pound_sign) ) 

      create_data_file(self.filename)
      write_data_in_file(self.filename)
      

In [49]:
# set of installed modules.
installed_modules = [str(module).replace(" ","==") for module in pkg_resources.working_set]
important_modules = [ 'tqdm', 'torch','tifffile', 'setuptools', 'scipy', 'scikit-learn', 'scikit-image', 'PyYAML', 'pysmb', 'pyfiglet', 'pip', 'Pillow', 'pandas', 'opencv-python-headless', 'numpy', 'numba', 'natsort', 'mrc', 'matplotlib', 'llvmlite', 'jupyter-core', 'jupyter-client', 'joblib', 'ipython', 'ipython-genutils', 'ipykernel', 'cellpose', 'big-fish']
filename = './metadata_'+ str(data_dir.name) +'.txt'

_, path_files, list_files_names, number_images = fa.ReadImages(data_dir).read()

def create_data_file(filename):
  if sys.platform == 'linux' or sys.platform == 'darwin':
    os.system('touch' + filename)
  elif sys.platform == 'win32':
    os.system('echo , > ' + filename)
    
    
number_spaces_pound_sign = 65

def write_data_in_file(filename):
    with open(filename, 'w') as fd:
        fd.write('#' * (number_spaces_pound_sign)) 
        fd.write('\n#       AUTHOR INFORMATION  ')
        fd.write('\n Author: ' + getpass.getuser())
        fd.write('\n Created at: ' + datetime.datetime.today().strftime('%d %b %Y'))
        fd.write('\n Time: ' + str(datetime.datetime.now().hour) + ':' + str(datetime.datetime.now().minute) )
        fd.write('\n Operative System: ' + sys.platform )
        fd.write('\n hostname: ' + socket.gethostname() + '\n')
        
        fd.write('#' * (number_spaces_pound_sign) ) 
        fd.write('\n#       PARAMETERS USED  ')
        
        fd.write('\n    channels_with_cytosol: ' + str(channels_with_cytosol) )
        fd.write('\n    channels_with_nucleus: ' + str(channels_with_nucleus) )
        fd.write('\n    channels_with_FISH: ' + str(channels_with_FISH) )
        fd.write('\n    diamter_nucleus: ' + str(diamter_nucleus) )
        fd.write('\n    diameter_cytosol: ' + str(diameter_cytosol) )
        fd.write('\n    voxel_size_z: ' + str(voxel_size_z) )
        fd.write('\n    voxel_size_yx: ' + str(voxel_size_yx) )
        fd.write('\n    psf_z: ' + str(psf_z) )
        fd.write('\n    psf_yx: ' + str(psf_yx) )
        fd.write('\n    minimum_spots_cluster: ' + str(minimum_spots_cluster) )
        fd.write('\n') 
        fd.write('#' * (number_spaces_pound_sign) ) 
        fd.write('\n#       FILES AND DIRECTORIES USED ')
        fd.write('\n Directory path: ' + str(data_dir) )
        fd.write('\n Folder name: ' + str(data_dir.name)  )
        # for loop for all the images.
        fd.write('\n Images in path :'  )
        
        for img_name in list_files_names:
          fd.write('\n    '+ img_name)
        fd.write('\n')  
        
        fd.write('#' * (number_spaces_pound_sign)) 
        fd.write('\n       REPRODUCIBILITY ')
        fd.write('\n Platform: \n')
        fd.write('    Python: ' + str(platform.python_version()) )
        fd.write('\n Dependancies: ')

        # iterating for all modules
        for module_name in installed_modules:
          if any(module_name[0:4] in s for s in important_modules):
            fd.write('\n    '+ module_name)
        fd.write('\n') 
        fd.write('#' * (number_spaces_pound_sign) ) 

create_data_file(filename)
write_data_in_file(filename)

In [50]:
with open(filename) as f:
    contents = f.read()
    print(contents)

#################################################################
#       AUTHOR INFORMATION  
 Author: luisaguilera
 Created at: 08 Nov 2021
 Time: 22:0
 Operative System: darwin
 hostname: Luis.local
#################################################################
#       PARAMETERS USED  
    channels_with_cytosol: [1, 2]
    channels_with_nucleus: 0
    channels_with_FISH: [1, 2]
    diamter_nucleus: 200
    diameter_cytosol: 250
    voxel_size_z: 500
    voxel_size_yx: 103
    psf_z: 600
    psf_yx: 150
    minimum_spots_cluster: 7
#################################################################
#       FILES AND DIRECTORIES USED 
 Directory path: /Users/luisaguilera/Desktop/FISH_Processing/dataBases/GAPDH_Exon_Cy5
 Folder name: GAPDH_Exon_Cy5
 Images in path :
    ROI001_XY1620755243_Z00_T0_merged.tif
    ROI002_XY1620755646_Z00_T0_merged.tif
    ROI003_XY1620755767_Z00_T0_merged.tif
    ROI004_XY1620755901_Z00_T0_merged.tif
    ROI005_XY1620756000_Z00_T0_merged.tif
    ROI006_